# Brazilian E-Commerce Case Data Processing

## Setup

In [ ]:
!pip install pandasql

In [ ]:
import pandas as pd
from pandasql import sqldf

setup data

In [ ]:
df_olist_customers_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_customers_dataset.csv")
df_olist_geolocation_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_geolocation_dataset.csv")
df_olist_order_items_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_order_items_dataset.csv")
df_olist_order_payments_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_order_payments_dataset.csv")
df_olist_order_reviews_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_order_reviews_dataset.csv")
df_olist_orders_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_orders_dataset.csv")
df_olist_products_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_products_dataset.csv")
df_olist_sellers_dataset = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/olist_sellers_dataset.csv")
df_product_category_name_translation = pd.read_csv("https://raw.githubusercontent.com/naufalhilmi12345/dataCode/refs/heads/master/Data%20Analysis/Brazilian%20E-Commerce%20Case/product_category_name_translation.csv")

## Data Preview

In [ ]:
print("df_olist_customers_dataset")
df_olist_customers_dataset.info()
print("\ndf_olist_geolocation_dataset")
df_olist_geolocation_dataset.info()
print("\ndf_olist_order_items_dataset")
df_olist_order_items_dataset.info()
print("\ndf_olist_order_payments_dataset")
df_olist_order_payments_dataset.info()
print("\ndf_olist_order_reviews_dataset")
df_olist_order_reviews_dataset.info()
print("\ndf_olist_orders_dataset")
df_olist_orders_dataset.info()
print("\ndf_olist_products_dataset")
df_olist_products_dataset.info()
print("\ndf_olist_sellers_dataset")
df_olist_sellers_dataset.info()
print("\ndf_product_category_name_translation")
df_product_category_name_translation.info()

df_olist_customers_dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB

df_olist_geolocation_dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  flo

## Make Flat Table

In [ ]:
df_flat_transaction = sqldf('''
  SELECT
    df_olist_order_items_dataset.order_id
    , df_olist_order_items_dataset.order_item_id
    , DATE(df_olist_order_items_dataset.shipping_limit_date) AS shipping_limit_date
    , df_olist_order_items_dataset.price
    , df_olist_order_items_dataset.freight_value
    , df_olist_orders_dataset.order_status
    , DATE(df_olist_orders_dataset.order_purchase_timestamp) AS order_purchase_date
    , strftime('%Y', df_olist_orders_dataset.order_purchase_timestamp)||'-'||strftime('%m', df_olist_orders_dataset.order_purchase_timestamp)||'-01' AS order_purchase_month
    , strftime('%Y', df_olist_orders_dataset.order_purchase_timestamp)||'-01-01' AS order_purchase_year
    , DATE(df_olist_orders_dataset.order_approved_at) AS order_approved_date
    , DATE(df_olist_orders_dataset.order_delivered_carrier_date) AS order_delivered_carrier_date
    , DATE(df_olist_orders_dataset.order_delivered_customer_date) AS order_delivered_customer_date
    , DATE(df_olist_orders_dataset.order_estimated_delivery_date) AS order_estimated_delivery_date
    , df_olist_products_dataset.product_category_name
    , df_olist_products_dataset.product_name_lenght
    , df_olist_products_dataset.product_description_lenght
    , df_olist_products_dataset.product_photos_qty
    , df_olist_products_dataset.product_weight_g
    , df_olist_products_dataset.product_length_cm
    , df_olist_products_dataset.product_height_cm
    , df_olist_products_dataset.product_width_cm
    , df_olist_order_payments_dataset.payment_sequential
    , df_olist_order_payments_dataset.payment_type
    , df_olist_order_payments_dataset.payment_installments
    , df_olist_order_payments_dataset.payment_value
    , df_olist_order_reviews_dataset.review_score
    , df_olist_order_reviews_dataset.review_comment_title
    , df_olist_order_reviews_dataset.review_comment_message
    , DATE(df_olist_order_reviews_dataset.review_creation_date) AS review_creation_date
    , DATE(df_olist_order_reviews_dataset.review_answer_timestamp) AS review_answer_date
    , df_olist_customers_dataset.customer_unique_id
    , df_olist_customers_dataset.customer_city
    , df_olist_customers_dataset.customer_state
    , df_olist_sellers_dataset.seller_city
    , df_olist_sellers_dataset.seller_state
  FROM df_olist_order_items_dataset
  LEFT JOIN df_olist_orders_dataset
    ON df_olist_order_items_dataset.order_id = df_olist_orders_dataset.order_id
  LEFT JOIN df_olist_customers_dataset
    ON df_olist_orders_dataset.customer_id = df_olist_customers_dataset.customer_id
  LEFT JOIN df_olist_products_dataset
    ON df_olist_order_items_dataset.product_id = df_olist_products_dataset.product_id
  LEFT JOIN df_olist_sellers_dataset
    ON df_olist_order_items_dataset.seller_id = df_olist_sellers_dataset.seller_id
  LEFT JOIN df_olist_order_payments_dataset
    ON df_olist_order_items_dataset.order_id = df_olist_order_payments_dataset.order_id
  LEFT JOIN df_olist_order_reviews_dataset
    ON df_olist_order_items_dataset.order_id = df_olist_order_reviews_dataset.order_id
''')

In [ ]:
df_flat_transaction.to_excel('flat_transaction.xlsx', index=False)